# Введение

## Web-запросы

### Постановка задачи

Начнем с основ автоматизированных веб-запросов на примере задачи получения курсов валют. Курс валют – полезная и регулярно обновляемая информация, но залезать каждый раз на сайт за ней – трудоёмко. Значит, вам нужен скрипт, который будет в удобном виде выгружать информацию по курсам валют.

Будем использовать сервис cbr-xml-daily.ru. Он возвращает информацию о курсах валют в структурированном формате JSON, и в дальнейшем мы расширим этот подход на произвольные html страницы.

Итак, нам необходима функция, возвращающая курс заданной валюты в двух форматах:

1. Только значение курса валюты.

2. Полную информацию о валюте: курс, название, номинал и другие характеристики.

### Протокол HTTP

Мы будем работать с веб-запросами, и для начала полезно будет вспомнить, как работает интернет.

Один компьютер (клиент) отправляет запрос в определённом формате другому компьютеру (серверу) и получает в ответ текст и мультимедийные файлы (картинки, видео). Обычно это происходит по протоколу http, который формализует это общение.

Запрос в http состоит из нескольких частей:

- адрес, по которому мы обращаемся (это то, что вы вводите в строке браузера, например, www.google.com);<br />
- техническая информация, вроде кукисов и метода запроса;<br />
- иногда ещё дополнительные данные, например, если вы сами загружаете картинку.

Ответ содержит:

- статус ответа: 200 для успешного ответа; 404, если адрес не найден и т.д.; полный список http статусов можно посмотреть здесь ;<br />
- текст в запрошенном формате (html, xml, json...) или мультимедийные файлы; <br />
- техническую информацию.

В протоколе HTTP запросы могут делаться с помощью одного из так называемых методов. Самые популярные — это GET и POST .

Метод GET просто получает текстовую информацию или мультимедийный файл по адресу. Вы как бы говорите: "Хочу получить страницу по адресу www.google.com" , и сервер вам отвечает: "200, ок, держи вот html". Это самый базовый метод.

Метод POST служит для отправки форм; помимо адреса он может "заворачивать" в себя дополнительные данные, вроде полей формы или картинок. Вы как бы "заворачиваете" посылку и отдаёте её почтальону. 

Обычно всю эту работу делает браузер, но мы можем делать эти же запросы из кода. Мы могли бы даже написать свой маленький браузер на python, но в этом модуле мы сосредоточимся на автоматическом сборе информации.

## Библиотека requests

Сначала разберемся, как получить информацию с внешнего сервиса. В стандартной библиотеке есть модуль urllib2, который может справиться с этой задачей. Однако большинство разработчиков используют стороннюю библиотеку requests, потому что в ней лучше реализованы большинство методов и код получается проще. Перед началом работы библиотеку потребуется установить с помощью вашего пакетного менеджера, например, с помощью следующей команды в консоли.

In [1]:
# python -m pip install requests  

Далее мы импортируем библиотеку и отправим запрос к сервису с помощью метода GET к сервису с курсами валют:

In [2]:
import requests  
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')  

In [3]:
response

<Response [200]>

Мы получили объект ответа, который содержит всю нужную нам информацию. По умолчанию на экран выводится HTTP-код ответа 200. Это означает, что запрос был корректным, и сервер отдал нам нужную информацию.

Код ответа в виде числа можно получить с помощью метода status_code:

In [4]:
print(response.status_code)  

200


### Читаем результат

Мы сделали запрос и получили ответ. Давайте теперь посмотрим, как считывать текст. 

Адрес, по которому мы обращались, возвращает результат в json формате. Эти данные уже лежат в атрибуте text в полученном ответе response:

In [5]:
print(response.text)

{
    "Date": "2020-08-22T11:30:00+03:00",
    "PreviousDate": "2020-08-21T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2020\/08\/21\/daily_json.js",
    "Timestamp": "2020-08-23T23:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 53.2185,
            "Previous": 52.7242
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 43.6138,
            "Previous": 43.4203
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 97.7822,
            "Previous": 96.4631
   

Вы можете посмотреть на полный текст, повторив запрос или на этой странице в вашем браузере: https://www.cbr-xml-daily.ru/daily_json.js

Сейчас текст хранится просто в строковой переменной. Далее мы можем превратить эту строку в словарь. Сделать это можно с помощью JSON-парсера python, либо воспользовавшись методом json, который уже встроен в объект ответа response:

In [6]:
currencies = response.json()  
print(currencies)  

{'Date': '2020-08-22T11:30:00+03:00', 'PreviousDate': '2020-08-21T11:30:00+03:00', 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2020/08/21/daily_json.js', 'Timestamp': '2020-08-23T23:00:00+03:00', 'Valute': {'AUD': {'ID': 'R01010', 'NumCode': '036', 'CharCode': 'AUD', 'Nominal': 1, 'Name': 'Австралийский доллар', 'Value': 53.2185, 'Previous': 52.7242}, 'AZN': {'ID': 'R01020A', 'NumCode': '944', 'CharCode': 'AZN', 'Nominal': 1, 'Name': 'Азербайджанский манат', 'Value': 43.6138, 'Previous': 43.4203}, 'GBP': {'ID': 'R01035', 'NumCode': '826', 'CharCode': 'GBP', 'Nominal': 1, 'Name': 'Фунт стерлингов Соединенного королевства', 'Value': 97.7822, 'Previous': 96.4631}, 'AMD': {'ID': 'R01060', 'NumCode': '051', 'CharCode': 'AMD', 'Nominal': 100, 'Name': 'Армянских драмов', 'Value': 15.2878, 'Previous': 15.2043}, 'BYN': {'ID': 'R01090B', 'NumCode': '933', 'CharCode': 'BYN', 'Nominal': 1, 'Name': 'Белорусский рубль', 'Value': 29.3407, 'Previous': 29.6269}, 'BGN': {'ID': 'R01100', 'NumCode': '9

Теперь данные лежат в словаре, и мы можем легко получать необходимые нам значения:

In [7]:
currencies['Valute']['UAH']  

{'ID': 'R01720',
 'NumCode': '980',
 'CharCode': 'UAH',
 'Nominal': 10,
 'Name': 'Украинских гривен',
 'Value': 27.0339,
 'Previous': 26.8088}

Данные получены, и задание выполнено. 

In [8]:
print(currencies['Valute']['CZK']['Name'])

Чешских крон


### Оформляем функцию

В завершение давайте оформим наши вычисления в отдельную функцию, которой будет удобно пользоваться. На вход она должна принимать два параметра:

1. Название валюты currency. Например, 'EUR' или 'USD'.

2. Формат ответа format. При значении 'full' будем отдавать все, что знаем о валюте. А при значении format = 'value' только значение ключа 'Value', т. е. курс: 69.0286.

Оформим наши требования в коде:

In [9]:
def exchange_rates(currency, format='full'):    
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']    
    data = response[currency]     
    if format == 'full':    
        return data      
    elif format == 'value':    
        return data['Value']

#### Задание 1

Напишите функцию currency_name, которая по ID валюты возвращает ее название на русском языке.

In [24]:
def currency_name(id):
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']   
    data = response    
  
    for currency in data:

        if data[currency]['ID'] == id:
            return data[currency]['Name']
        

    


currency_name('R01700J')
    
    

'Турецкая лира'

#### HTML-страницы

##### Постановка задачи

Довольно часто приходится добывать информацию не из удобно форматированного json-файла, а прямо с HTML-страниц. Получить содержимое страницы в большинстве случаев несложно, труднее извлечь из HTML-кода нужную информацию. В качестве примера мы рассмотрим страницу новости, из которой будем доставать полезную информацию: 1) заголовок страницы; 2) дату публикации; 3) текст публикации; 4) ссылки на странице.

Получаем данные

Получить html страницу можно так же, как мы получали до этого json: используем библиотеку requests и метод GET:  

In [25]:
import requests  
  
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
  
response = requests.get(url)  
  
# Убедимся, что мы успешно получили ответ     
print(response.status_code)    
# => 200   
  
# Выведем полученные данные    
print(response.text)   

200
<!doctype html>
<html class="no-js bg-fixed _no-bg" style="background-image:url(https://nplus1.ru/images/2019/06/04/b32b62189fb87cce895e229e1d6d27b4.jpeg)" lang="">
<head>
    
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
            
    <link rel="canonical" href="https://nplus1.ru/news/2019/06/04/slothbot" />

        <title>Робота-ленивца научили лазать по паутине из тросов</title>

    	    <meta itemprop="datePublished" content="2019-06-04"/>
	
	    <meta name="mediator_author" content="Григорий Копиев"/> 
	
        <!-- amp page -->
    <link rel="amphtml" href="https://nplus1.ru/news/2019/06/04/slothbot/amp">
    

        <!-- for Google -->
        <meta name="description" content="Он сам добывает энергию с помощью солнечных п

Мы получили большую строку с текстом в формате html, который используется для визуальной разметки. Это позволяет делать информацию более наглядной для людей, но в отличие от json, мы не можем просто преобразовать его напрямую в словарь. Далее мы посмотрим, как извлекать информацию из подобных страниц

## Библиотека BeautifulSoup

Для поиска данных на странице воспользуемся библиотекой BeautifulSoup. Она позволяет по названию тэгов и их атрибутов получать содержащийся в них текст.

Она не является частью стандартной библиотеки, поэтому для начала её нужно установить с помощью пакетного менеджера, например, введя в консоли:

In [26]:
# pip install beautifulsoup4  

Теперь мы можем получать данные из страницы. Давайте получим title (это строка, отображающая в браузерах на закладках)

In [29]:
# Импортируем наши библиотеки    
from bs4 import BeautifulSoup    
import requests    
    
# Получаем данные, как и ранее    
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
response = requests.get(url)    
    
# Теперь создадим объект BeautifulSoup, указывая html парсер    
page = BeautifulSoup(response.text, 'html.parser')

# Всё готово, чтобы получать данные из страницы    
# Для начала получим title, отображающийся на закладках браузера    
print(page.title)  
# => <title>Робота-ленивца научили лазать по паутине из тросов</title>  
    
# Мы получили тэг. Чтобы достать из него текст, вызовем атрибут text    
page.title.text    
# => 'Робота-ленивца научили лазать по паутине из тросов'  

<title>Робота-ленивца научили лазать по паутине из тросов</title>


'Робота-ленивца научили лазать по паутине из тросов'

### Данные со страницы

#### Читаем заголовок

Мы получили данные из title, но как нам забрать текст с самой страницы? Мы можем сделать это, запросив содержимое определённых тэгов. 

Пусть мы знаем, что заголовок статьи находится в тэге h1. Тогда мы можем получить его текст с помощью метода find

In [30]:
print(page.find('h1').text)

Робота-ленивца научили лазать по паутине из тросов


Как узнать, какой именно тэг нужен? Проще всего это сделать с помощью "инструментов разработчика", которые есть во всех современных браузерах.

In [31]:
print(page.find('time').text)   


18:27
04 Июнь 2019



#### Задание 2 - Автоматический сбор заголовков

Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок для статей на википедии

In [34]:
def wiki_header(url):
    
    # Получаем данные, как и ранее    
    response = requests.get(url)    

    # Теперь создадим объект BeautifulSoup, указывая html парсер    
    page = BeautifulSoup(response.text, 'html.parser')

    # Теперь вернем заголовок
    return page.find('h1').text



wiki_header('https://en.wikipedia.org/wiki/Operating_system')

'Operating system'

#### Больше данных

Мы рассмотрели базовый поиск конкретного элемента. Могут встречаться другие вариации:

- когда нам нужно получить много похожих элементов;
- когда тэг атрибута неуникальный, тогда нам нужно использовать атрибуты (id, class), либо использовать вложенность.

Посмотрим на практике, что делать в случае, когда тэг элемента неуникальный. Пусть мы хотим получить сам текст статьи. Мы видим, что он находится в тэге div

Тэги div очень распространённые, их много на странице. Если мы просто используем find, то получим первый попавшийся, но это не то, что нам надо.

Тут мы можем заметить, что у искомого текста есть свой класс GeneralMaterial-body. Воспользуемся этим:  в метод find можно передать аргумент class_

In [36]:
print(page.find('div', class_='body').text)  







Allison Carter, Georgia Tech





Американские инженеры разработали робота-ленивца, способного лазать по тросам и перемещаться с одного троса на другой. Благодаря относительно малому потреблению энергии и использованию солнечных панелей подобного робота можно практически неограниченное время использовать для наблюдений на деревьях в лесу, рассказывают авторы статьи, представленной на конференции ICRA 2019.
Создание роботов, способных лазать по тросам — важная технологическая задача, наработки из которой потенциально применимы во многих областях. К примеру, в 2016 году в России создали робота для проверки высоковольтных линий электропередачи. Дрон спускает такого робота на грозозащитный трос, после чего тот самостоятельно перемещается вдоль основных проводов и осматривает их. Кроме того, потенциально таких роботов можно применять для исследований в густых лесах, в которых ветки соседних деревьев соприкасаются.
Инженеры из Технологического института Джорджии под руководством Магнус

Отлично, сработало. Аналогично работает и с id, мы могли бы написать что-то вроде `page.find('div', id='maincontent')

#### Задание 1. Поиск элемента

У вас есть переменная page, в которой хранится содержимое html-страницы. На странице есть элемент в тэге span, у которого атрибут id равен 'target'. Напишите строчку кода, которая присвоит текст этого элемента переменной value.

Запишите ваш код в одну строку без пробелов. Используйте апострофы для передачи параметров, содержащих значение тэга и его id.

In [37]:
value = page.find('span', id='target').text

AttributeError: 'NoneType' object has no attribute 'text'

#### Сбор нескольких элементов

Рассмотрим случай, когда нам нужно сразу много элементов. Пусть мы хотим получить название всех ссылок на странице в википедии про языки программирования.

Для ссылок существует тэг <a\><\/a\>.  Давайте попробуем использовать find  

In [38]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'  
  
response = requests.get(url)  
page = BeautifulSoup(response.text, 'html.parser')  
page.find('a')  
# => <a id="top"></a>  

<a id="top"></a>

Что-то пошло не так, и мы получили только одну ссылку, хотя на странице их явно больше. Это происходит, потому что метод find возвращает только первый подходящий элемент. Если нам надо их больше, нужно воспользоваться методом find_all

In [40]:
links = page.find_all('a')  
# Посмотрим, сколько всего мы получили  
print(len(links))  
# => 937
  
# Посмотрим на некоторые из ссылок  
print([link.text for link in links[500:510]])  
# => ['Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal']

927
['Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal']


Ещё одна полезная вещь:  последовательный поиск, т.е. мы можем найти сначала один элемент, а потом сделать внутри него второй поиск. Давайте выведем названия всех ссылок для языков программирования, которые начинаются на цифру "1".

In [46]:
# Получаем элемент первого блока со ссылками  
first_block = page.find('div', class_='div-col')  
  
# Берём оттуда ссылки (ограничимся первыми десятью)  
links = first_block.find_all('a') 
print(links)
print([link.text for link in links[:10]])  
# => ['1C:Enterprise programming language']  

[<a href="/wiki/A_Sharp_(.NET)" title="A Sharp (.NET)">A.NET</a>, <a href="/wiki/A-0_System" title="A-0 System">A-0 System</a>, <a href="/wiki/A%2B_(programming_language)" title="A+ (programming language)">A+</a>, <a href="/wiki/ABAP" title="ABAP">ABAP</a>, <a href="/wiki/ABC_(programming_language)" title="ABC (programming language)">ABC</a>, <a href="/wiki/ABC_ALGOL" title="ABC ALGOL">ABC ALGOL</a>, <a href="/wiki/ACC_(programming_language)" title="ACC (programming language)">ACC</a>, <a class="mw-redirect" href="/wiki/Accent_(programming_language)" title="Accent (programming language)">Accent</a>, <a href="/wiki/Distributed_Application_Specification_Language" title="Distributed Application Specification Language">Ace DASL (Distributed Application Specification Language)</a>, <a href="/wiki/Action!_(programming_language)" title="Action! (programming language)">Action!</a>, <a href="/wiki/ActionScript" title="ActionScript">ActionScript</a>, <a href="/wiki/Actor_(programming_language)" 

В заключение заметим, что BeautifulSoup – достаточно мощная библиотека. Мы рассмотрели базовые возможности, но полный список гораздо шире. С ним можно ознакомиться в официальной документации: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

#### Задание 1 - Актёры

Напишите функцию get_actors, которая по ссылке на страницу фильма на кинопоиске возвращает список актёров из колонки справа: https://www.kinopoisk.ru/top/lists/

NB: Если делать слишком часто запросы, кинопоиск начинает выдавать капчу, поэтому старайтесь при отладке не делать больше одного запроса в секунду

In [47]:
def get_actors(url):
    list_actors=[]
    response=requests.get(url)
    page=BeautifulSoup(response.text,'html.parser')
    actors=page.find_all('li',itemprop='actors')
    for actor in actors:
        list_actors.append(actor.text)
    return list_actors


get_actors('https://www.kinopoisk.ru/film/42326/')

ConnectionError: HTTPSConnectionPool(host='www.kinopoisk.ru', port=443): Max retries exceeded with url: /film/42326/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001A7DA2C76C8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

## HTML-таблицы

### Постановка задачи

При работе с web-страницами было бы здорово получать содержимое  таблиц в виде датафрейма. Рассмотрим страницу ключевых показателей Центрального банка РФ cbr.ru. Наша задача будет состоять в том, чтобы получить одну из таблиц виджетов в виде датафрейма. Например, таблицу цен на драгоценные металлы.

Метод read_html из pandas умеет автоматически находить на HTML-странице таблицы и возвращать их списком из датафреймов:

In [50]:
import pandas as pd  
url = 'https://www.cbr.ru/key-indicators/'  
# Таблица с драгметаллами оказалась второй по счёту  
pd.read_html(url)[0]  

,0,1,2
0,валюта,21.08.2020,22.08.2020
1,Доллар США USD,737711,740999
2,Евро EUR,874261,877343


Если этого не происходит, но вы уверены в том, что порядок таблиц на странице неизменен, то можно вручную найти нужную таблицу на странице. Этому случаю посвящены следующие шаги.

### Поиск таблицы с помощью BeautifulSoup

Более надёжным решением является сначала получить только текст таблицы и передавать в pandas только её. Так же, как и в предыдущем разделе, используем requests и BeautifulSoup, чтобы получить данные со страницы.

А теперь — внимание! В случае работы с сайтом Центробанка нам нужно будет учесть ещё один нюанс. С недавнего времени администраторы стали блокировать обращения к сайту, поступающие не от «живых» пользователей, а от компьютерных программ (в том числе от той, которую мы собираемся написать прямо сейчас). Есть ли шанс в таких условиях всё-таки получить данные, не открывая страницу в браузере?

Безусловно, это возможно. Безвыходных положений вообще не бывает, особенно в программировании! Один из способов справиться с ситуацией — «притвориться» браузером :).

Каждый запрос, отправляемый с помощью команды get, имеет заголовок. Если заголовок не задан явно, то он создаётся автоматически, и туда добавляется информация в том числе о том, кто является субъектом (или источником) запроса — человек или программа. Чтобы «притвориться» браузером, нам просто нужно вручную дописать нужные сведения в этот заголовок. Подробнее о таком подходе можно почитать в этой статье.

Итоговый вариант кода после добавления заголовка должен выглядеть так:

In [51]:
from bs4 import BeautifulSoup  
import pandas as pd  
import requests  
  
url = 'https://www.cbr.ru/key-indicators/'
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')  

Далее нажимаем 'Исследовать элемент' в меню браузера на виджете с драгоценными металлами:

Нам необходимо добраться до кода таблицы, который начинается с тэга <\table>. Таблиц на странице много, поэтому указываем путь к таблице виджета драгоценных металлов применяя знания полученные ранее:

In [52]:
all_blocks = soup.find_all('div', class_='key-indicator_content offset-md-2')
# Данные таблицы с драгметаллами находятся во втором блоке
data = all_blocks[1].find('table') 

Сейчас наша таблица записана в HTML-коде:

In [53]:
print(data)

<table>
<tbody>
<tr class="denotements">
<td class="value td-w-13">рублей за грамм</td>
<td class="value td-w-4 _end">22.08.2020</td>
</tr>
<tr>
<td class="value td-w-9 _inner">
<div class="d-flex title-subinfo">
<div class="col-md-5">Золото</div>
<div class="col-md-3 offset-md-1 _subinfo">Au</div>
</div>
</td>
<td class="value td-w-4 _bold _end mono-num">4 604,75</td>
</tr>
<tr>
<td class="value td-w-9 _inner">
<div class="d-flex title-subinfo">
<div class="col-md-5">Серебро</div>
<div class="col-md-3 offset-md-1 _subinfo">Ag</div>
</div>
</td>
<td class="value td-w-4 _bold _end mono-num">63,97</td>
</tr>
<tr>
<td class="value td-w-9 _inner">
<div class="d-flex title-subinfo">
<div class="col-md-5">Платина</div>
<div class="col-md-3 offset-md-1 _subinfo">Pt</div>
</div>
</td>
<td class="value td-w-4 _bold _end mono-num">2 172,72</td>
</tr>
<tr>
<td class="value td-w-9 _inner">
<div class="d-flex title-subinfo">
<div class="col-md-5">Палладий</div>
<div class="col-md-3 offset-md-1 _sub

Для перевода ее в датафрейм используем метод read_html. Метод возвращает список датафреймов. В нашем случае датафрейм будет только один, поэтому сразу берем первый элемент:

In [54]:
df = pd.read_html(str(data))[0]  
df  

,0,1
0,рублей за грамм,22.08.2020
1,Золото Au,"4 604,75"
2,Серебро Ag,6397
3,Платина Pt,"2 172,72"
4,Палладий Pd,"5 193,56"


Датафрейм получен! Теперь можно работать с ним средствами Pandas.

#### Задание 2 - Рейтинг банков

Напишите программу, которая забирает данные из таблицы рейтинга банков с https://www.banki.ru/banks/ratings/, делает из него датафрейм и сохраняет его в переменую df.

In [68]:
url = 'https://www.banki.ru/banks/ratings/'
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')
# Данные таблицы с банками находятся в первом блоке
data = soup.find('table')
df = pd.read_html(str(data), header=None)[0]
df.columns = ['rating','name','from',
                     'to','diff','diff %']
df




,rating,name,from,to,diff,diff %
0,1,"Сбербанк России лицензия № 1481, Москва и обл.",31 415 098 095,30 759 269 658,+655 828 437,"+2,13%"
1,2,"ВТБ лицензия № 1000, Санкт-Петербург и обл.",15 269 639 853,15 029 152 247,+240 487 606,"+1,60%"
2,3,"Газпромбанк лицензия № 354, Москва и обл.",7 031 457 276,7 092 861 002,−61 403 726,"−0,87%"
3,4,Национальный Клиринговый Центр лицензия № 3466...,4 487 642 224,4 279 004 841,+208 637 383,"+4,88%"
4,5,"Альфа-Банк лицензия № 1326, Москва и обл.",3 937 290 975,3 908 179 939,+29 111 036,"+0,74%"
5,6,"Россельхозбанк лицензия № 3349, Москва и обл.",3 619 545 036,3 506 105 175,+113 439 861,"+3,24%"
6,7 +1,"Московский Кредитный Банк лицензия № 1978, Мос...",2 841 108 461,2 710 108 949,+130 999 512,"+4,83%"
7,8 −1,"Банк Открытие лицензия № 2209, Москва и обл.",2 834 457 224,2 733 425 636,+101 031 588,"+3,70%"
8,9,"Совкомбанк лицензия № 963, Костромская обл.",1 574 405 911,1 504 044 466,+70 361 445,"+4,68%"
9,10,"ЮниКредит Банк лицензия № 1, Москва и обл.",1 447 538 742,1 473 088 658,−25 549 916,"−1,73%"


Возможно, кто-то обратил внимание, что когда мы "стягиваем" таблицу стоимости металлов, то значение стоимости серебра (самого дешёвого из представленных металлов) может превосходить стоимость более дорогих металлов. Это связано  с отсутствием указания разделителей тысяч и дробной части.

df = pd.read_html(str(data), thousands =" ", decimal = ",")[0]  может исправить эту ситуацию.

## Введение в API

В предыдущих разделах мы собирали полезную информацию с различных сайтов с выделением правильных тэгов – это довольно трудоёмкая задача. Кроме того, подобные программы могут ломаться в случаях, когда меняется дизайн сайта, его разметка, или владельцы сайтов защищаются от ботов капчей. 

К счастью, многие крупные сайты предоставляют доступ к так называемым API (application programming interface, программный интерфейс приложения). Это специальные разделы сайта, где информацию можно получать без разметки, формат запросов – ответов зафиксирован, и они созданы специально, чтобы облегчить жизнь сторонним разработчикам.

Мы уже видели, как https://www.cbr-xml-daily.ru возвращает данные о валютах в json-формате, и это может считаться API. В этом разделе мы на примере ВКонтакте посмотрим особенности API, характерные для более крупных сайтов.

#### Сервисный токен

Для того чтобы начать работать с API ВКонтакте, необходимо получить сервисный токен (ключ) авторизации. 

Авторизация применяется практически во всех API, чтобы отдавать данные только их владельцу или контролировать количество запросов в единицу времени. Сервисный токен для нашей задачи создается вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

Зайдите на страницу https://vk.com/editapp?act=create, чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название и оставьте значение платформы "Standalone-приложение":

После подтверждения создания приложения в приложении ВК или по СМС зайдите в настройки:

#### Запрос к API

Для примера работы с API мы будем получать данные для статистических отчетов произвольной группы, например:

- соотношение мужчин и женщин в группе;
- статистика географии пользователей;
- другие данные для аналитики групп конкурентов.

Сначала рассмотрим работу API на простом примере, на основе которого работают многие системы. 

Перейдите по следующей ссылке в браузере, подставив вместо слова "TOKEN" сервисный токен из прошлого шага:

https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN

Результат:

In [71]:
# {"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","is_closed":false,"can_access_closed":true}]}

Сейчас мы сделали GET-запрос к API ВКонтакте, который состоит из следующих частей:

- https://api.vk.com/method — домен и URL запроса API: обычно не меняется; 
- users.get — название метода, который отдает определенный отчет; нашем случае это метод для получения информации о пользователе; 
- user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию, и номер версии API; 
- token, который выдается только пользователям, имеющим право просматривать определенные данные, например, показания счетчиков - Яндекс, метрики вашего проекта: на все остальные запросы без корректного токена система отвечает отказом.

Если мы посмотрим документацию метода users.get, то увидим, что в ней описано множество других параметров, которые можно получить о пользователе: дата рождения, пол, родной город и другие.

Словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал). Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

In [72]:
# https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN

Результат:

In [73]:
# {"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","is_closed":false,"can_access_closed":true,"sex":2,"bdate":"10.10.1984"}]}  
# Примечание: значение 2 у параметра 'sex' означает мужской пол.  

#### Запрос к API из кода

Мы делали запрос в браузере, теперь давайте сделаем запрос из кода. Будем пользоваться всё той же библиотекой requests.

In [78]:
url = 'https://api.vk.com/method/users.get'
token = 'b999d26bb999d26bb999d26bc6b9ea49cebb999b999d26be6da711b86f2c898d1bb0150'
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}  
  
# Мы можем выставить параметры запроса через аргумент params  
response = requests.get(url, params=params)  
response.text  
# => '{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","is_closed":false,"can_access_closed":true,"sex":2,"bdate":"10.10.1984"}]}'

'{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","is_closed":false,"can_access_closed":true,"sex":2,"bdate":"10.10.1984"}]}'

Мы получили строку в формате JSON. Как мы помним по первому разделу, её можно преобразовать в словарь методом json и после этого обращаться к различным полям. Кроме того, такие большие вложенные словари нагляднее выводить с помощью функции pprint (~pretty print, красивый вывод), которым мы и воспользуемся

In [79]:
from pprint import pprint  
  
pprint(response.json())  
# =>   
# {'response': [{'bdate': '10.10.1984',  
#                'can_access_closed': True,  
#                'first_name': 'Павел',  
#                'id': 1,  
#                'is_closed': False,
#                'last_name': 'Дуров',  
#                'sex': 2}]}  
  
user = response.json()['response'][0]  
  
# Выведем дату рождения  
print(user['bdate'])  
# => 10.10.1984  
  
# Выведем имя  
print(user['first_name'])  
# => Павел  

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}
10.10.1984
Павел


Данный метод позволяет запрашивать сразу много пользователей (до 1000).  Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате, например: '1,2,3'.

In [81]:
ids = ",".join(map(str, range(1, 4)))  
print(ids)  
# => 1,2,3  
  
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': token, 'lang': 'ru'}  
  
pprint(requests.get(url, params=params).json()) 
# {'response': [{'can_access_closed': True,  
#                'first_name': 'Павел',  
#                'id': 1,  
#                ''is_closed': False',  
#                'last_name': 'Дуров'},  
#               {'can_access_closed': False,  
#                'first_name': 'Александра',  
#                'id': 2,  
#                'is_closed': True,  
#                'last_name': 'Владимирова'},  
#               {'deactivated': 'deleted',  
#                'first_name': 'DELETED',  
#                'id': 3,  
#                'last_name': ''}]}

1,2,3
{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


#### Задание 1 - Гендерный баланс

Используя API, определите долю женщин(sex=1) среди пользователей с id от 1 до 500. Иногда вам будут попадать пользователи, у которых пол не указан (sex=0), их не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после запятой, например, 0.55

Пример: если у нас будет 300 пользователей sex=1, 100 пользователей с sex=2 и 100 пользователей с sex=0, то в ответе должно быть 0.75

In [140]:
ids = ",".join(map(str, range(1, 501)))

params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': token, 'lang': 'ru'}

user = requests.get(url, params=params).json()['response']  

users_sex = {}

for i in range(0, 500):

    users_sex.update({i: [user[i]['id'], user[i]['sex']]})
    
df = pd.DataFrame.from_dict(users_sex, orient='index',
                       columns=['id', 'sex'])

print(df['sex'].value_counts())

2    218
1    203
0     79
Name: sex, dtype: int64
500


#### Ограничения API

В прошлом блоке мы собрали информацию о небольшом наборе пользователей. Теперь перейдем к более реальной задаче — сбору данных о пользователях группы ВКонтакте. 

Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:

- не могут учитывать всех особенностей вашего проекта;
- считают фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
- не всегда бесплатны и вряд ли позволят работать с большими объемами данных.

В этом разделе мы научимся считать произвольные метрики групп, собирая данные из API. 

В текущем шаге мы научимся работать с двумя ограничениями, которые свойственны практически всем системам:

- ограничение на количество вызовов в единицу времени;
- ограничение на количество выгружаемых строк за один запрос.

Ограничение на количество запросов в секунду сделано для того, чтобы избежать чрезмерной нагрузки на серверы системы. В ряде случаев небольшое количество отчетов можно выгрузить, уложившись в этот лимит (например, как мы получили информацию о 500 пользователях в прошлом упражнении). Однако второе ограничение не удастся «обойти» в случае выгрузки больших отчетов. Например, чтобы получить список всех пользователей очень популярной группы, серверу, возможно, придется отправить вам разом лист из миллионов записей.

Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/mountelbrus.

Посмотрим в документации, какие методы нам доступны для групп. Для получения списка пользователей группы видим метод groups.getMembers.

Согласно документации, обязательным параметром является ID или короткое имя группы. В нашем случае это mountelbrus: https://vk.com/mountelbrus. Тестируем, как работает метод в самом простом случае:

In [146]:
url = 'https://api.vk.com/method/groups.getMembers'  
params = {  
    'group_id': 'mountelbrus',  
    'v': 5.95,  
    'access_token': token  
}  
response = requests.get(url, params = params)  
data = response.json()  
print(data)  
# => {'response': {'count': 65921, 'items': [161, 273, 710, 1157...  

 
len(data['response']['items'])  
# => 1000     

{'response': {'count': 66145, 'items': [161, 273, 710, 1157, 1364, 1858, 2306, 3516, 3857, 4316, 4592, 4646, 4765, 4901, 4969, 4992, 5118, 5121, 5438, 5453, 5678, 5818, 5970, 6172, 6528, 6706, 7224, 7303, 7638, 8212, 8376, 8422, 9006, 9322, 9640, 9808, 10133, 10361, 10531, 11246, 11788, 12129, 12541, 12757, 12915, 12976, 12990, 13188, 13549, 13695, 14033, 14644, 14742, 15018, 15140, 15421, 16328, 16608, 17522, 17656, 17670, 18226, 18248, 18423, 18958, 19072, 19232, 19405, 19483, 19568, 19691, 19720, 20417, 20598, 20866, 20934, 20980, 21179, 21386, 21760, 22123, 22135, 22726, 23011, 23217, 23256, 23360, 23932, 24275, 24985, 25018, 25096, 25166, 25310, 25394, 25814, 25930, 25975, 26034, 26205, 26543, 26828, 27023, 27513, 27536, 27659, 27670, 28879, 29166, 29450, 29677, 29805, 30168, 30518, 30760, 31012, 31926, 32019, 32528, 33259, 33299, 33795, 34777, 34811, 35170, 35244, 35300, 35868, 36811, 37082, 37311, 37455, 37496, 38193, 38258, 39030, 39286, 39425, 39538, 39702, 39737, 39738, 40089

IndexError: list index out of range

Мы видим, что всего пользователей в группе больше 65 000, а получили мы только первую тысячу пользователей группы. Судя по описанию параметра count в документации, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей воспользуемся параметрами count и offset: будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), а в каждом следующем шаге цикла увеличим смещение offset на величину count.

Сначала напишем цикл выгрузки первых 20 пользователей со значением count = 5. Т.е. цикл будет выгружать за 1 запрос по 5 пользователей, пока не выгрузит первые 20. Это позволит нам проверить корректность работы цикла перед тем, как делать большие и долгие выгрузки.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки с 1000 пользователей, чтобы было с чем сверить результат выгрузки из 20 пользователей:

In [142]:
users_for_checking = data['response']['items'][:20]  
print(users_for_checking)  

[161, 273, 710, 1157, 1364, 1858, 2306, 3516, 3857, 4316, 4592, 4646, 4765, 4901, 4969, 4992, 5118, 5121, 5438, 5453]


Теперь используем count и offset, чтобы получить те же id по 5 за раз

In [144]:
count = 1000  
offset = 0  
user_ids = []
max_count = data['response']['count']  
while offset < max_count:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {  
        'group_id': 'mountelbrus',  
        'v': 5.95,  
        'count': count,  
        'offset': offset,  
        'access_token': token  
    }     
    # такой же запрос как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']   
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
      
# Выгружаю 5 пользователей с offset = 0  
# Выгружаю 5 пользователей с offset = 5  
# Выгружаю 5 пользователей с offset = 10  
# Выгружаю 5 пользователей с offset = 15  
print(user_ids)  
# => [161, 273, 710, 1157, 1364, 1858, 2306, 3516, 3857, 4316, 4592, 4646, 4765, 4901, 4969, 4992, 5118, 5121, 5438, 5453]  
  
user_ids == users_for_checking  
# => True  

Выгружаю 1000 пользователей с offset = 0
Выгружаю 1000 пользователей с offset = 1000
Выгружаю 1000 пользователей с offset = 2000
Выгружаю 1000 пользователей с offset = 3000
Выгружаю 1000 пользователей с offset = 4000
Выгружаю 1000 пользователей с offset = 5000
Выгружаю 1000 пользователей с offset = 6000
Выгружаю 1000 пользователей с offset = 7000
Выгружаю 1000 пользователей с offset = 8000
Выгружаю 1000 пользователей с offset = 9000
Выгружаю 1000 пользователей с offset = 10000
Выгружаю 1000 пользователей с offset = 11000
Выгружаю 1000 пользователей с offset = 12000
Выгружаю 1000 пользователей с offset = 13000
Выгружаю 1000 пользователей с offset = 14000
Выгружаю 1000 пользователей с offset = 15000
Выгружаю 1000 пользователей с offset = 16000
Выгружаю 1000 пользователей с offset = 17000
Выгружаю 1000 пользователей с offset = 18000
Выгружаю 1000 пользователей с offset = 19000
Выгружаю 1000 пользователей с offset = 20000
Выгружаю 1000 пользователей с offset = 21000
Выгружаю 1000 пользоват

False

См. статью "Получение участников сообщества vk.com за считанные секунды" https://habr.com/ru/post/248725/

Видим, что подход корректно работает. Теперь мы можем получить всех пользователей, выставив count = 1000 и max_count = data['response']['count'].

#### Задание 1 - Юбилейный пользователь

Используя API, определите id пятидесятитысячного пользователя в группе mountelbrus . Т.е. если бы у нас не было ограничения на размер запроса, то мы бы запросили data['response']['items'][49999] .

In [147]:
user_ids[49999]

225379369

#### Ограничение по частоте запросов

В API частот добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали  сервер. Подобное ограничение есть и у ВКонтакте, в документации которой указано, что можно делать не более 3-х запросов в секунду. 

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, т.к. периодичность отправки запросов будет искусственно замедлена. Воспользуемся библиотекой time и методом sleep (пауза в 0.5 секунды добавлена в конце цикла):

In [148]:
import time  
  
count = 1000  
offset = 0  
user_ids = []  
while offset < 10000:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {  
        'group_id': 'mountelbrus',  
        'v': 5.95,  
        'count': count,  
        'offset': offset,  
        'access_token': token  
    }    
    # такой же запрос, как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']    
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
    
    print('Ожидаю 0.5 секунды...')  
    time.sleep(0.5)  

Выгружаю 1000 пользователей с offset = 0
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 1000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 2000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 3000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 4000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 5000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 6000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 7000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 8000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 9000
Ожидаю 0.5 секунды...


#### Лайки, репосты и комментарии

Ещё одна полезная вещь, которую можно получить,  —  это количество взаимодействий с постами через API новостной ленты. На данном этапе будем использовать самый простой вариант: берем последние 10 постов группы. Мы продолжаем работать с группой https://vk.com/mountelbrus.

Начнем с  формирования запроса к API ВКонтакте методом wall.get():

In [149]:
import requests  
from pprint import pprint  
  
url = 'https://api.vk.com/method/wall.get'  
params = {  
    'domain': 'mountelbrus',  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
}  
response = requests.get(url, params = params)  
response.json()
# => {'response': {'count': 705,  
#      'items': [{'id': 13401',  
#        'from_id': -9690799,  
#        'owner_id': -9690799,  
#        'date': 1596525625,  
#        'marked_as_ads': 0,  
#        'post_type': 'post',
# ... 

{'response': {'count': 710,
  'items': [{'id': 13482,
    'from_id': -9690799,
    'owner_id': -9690799,
    'date': 1598188550,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Последние 10 туров на Эльбрус в этом сезоне, которые еще доступны для бронирования. \nРасписание 2020, описание маршрутов и бронирование здесь: https://strahu-net.com/raspisanie-turov-na-elbrus/?year=2020',
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1598188543,
       'id': 457242581,
       'owner_id': -9690799,
       'has_tags': False,
       'access_key': '04ffc873146b0f0bf5',
       'post_id': 13482,
       'sizes': [{'height': 65,
         'url': 'https://sun9-56.userapi.com/2jgpYP7K7Bn0KUQRBrBs5sTiNVYS8oNyDJwBwQ/-NQdNzlSXTI.jpg',
         'type': 'm',
         'width': 130},
        {'height': 87,
         'url': 'https://sun9-2.userapi.com/IPbtbVfkd3KdYy41z6eR2CJJqY79FJgdn8bATw/ghCgd2y6Ymk.jpg',
         'type': 'o',
         'width': 130},
        

Видим, что сначала идёт общее количество постов, а по ключу 'items'  —  сами посты. Посмотрим на отдельный пост:

In [150]:
response.json()['response']['items'][0]  
# {'id': 13401,  
#  'from_id': -9690799,  
#  'owner_id': -9690799,  
#  'date': 1596525625,  
#  'marked_as_ads': 0,  
#  'post_type': 'post',  
#  'text': 'Live: 4.08.2020 Инструктор, мастер спорта по альпинизму Гергель Алексей проводит брифинг перед походом на Казбек',  
# ...  
#    'title': 'Live: Восхождение на Эльбрус | Elbrus Climbing',  
#    'track_code': 'video_a01b60fecgAaGKK4Ixj13XSddJY5HBUZCTQcBUaoa1vY2ZWYaHRJK2gQp7UjHszkK6lBoAsvLCkxBQ',  
#    'views': 8848}}],  
#  'post_source': {'type': 'vk'},  
#  'comments': {'count': 16, 'can_post': 1, 'groups_can_post': True},  
#  'likes': {'count': 41, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},  
#  'reposts': {'count': 3, 'user_reposted': 0},  
#  'views': {'count': 5837},
#  'edited': 1596530499} 

{'id': 13482,
 'from_id': -9690799,
 'owner_id': -9690799,
 'date': 1598188550,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Последние 10 туров на Эльбрус в этом сезоне, которые еще доступны для бронирования. \nРасписание 2020, описание маршрутов и бронирование здесь: https://strahu-net.com/raspisanie-turov-na-elbrus/?year=2020',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1598188543,
    'id': 457242581,
    'owner_id': -9690799,
    'has_tags': False,
    'access_key': '04ffc873146b0f0bf5',
    'post_id': 13482,
    'sizes': [{'height': 65,
      'url': 'https://sun9-56.userapi.com/2jgpYP7K7Bn0KUQRBrBs5sTiNVYS8oNyDJwBwQ/-NQdNzlSXTI.jpg',
      'type': 'm',
      'width': 130},
     {'height': 87,
      'url': 'https://sun9-2.userapi.com/IPbtbVfkd3KdYy41z6eR2CJJqY79FJgdn8bATw/ghCgd2y6Ymk.jpg',
      'type': 'o',
      'width': 130},
     {'height': 133,
      'url': 'https://sun9-71.userapi.com/GALNTGZAbfiizVPaGERbJmOrUw5c84TTpHAS0g/0b0kiSnKV2I.

Нужная нам статистика находится в полях 'comments', 'likes' и 'reposts'. Соберем итоговую статистику для каждого поста в словарь stats. В качестве ключа будем использовать начало статьи, в качестве значения — список с тремя интересующими нас метриками и временем публикации: [комментарии, лайки, репосты, дата публикации]

In [151]:
stats = {}  
              
for record in response.json()['response']['items'][:]:  
    title = record['text'][:30]  
    if title:  
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date'] ]  
pprint(stats)  
  
# => {'7 ГОРЯЩИХ ТУРОВ НА ЭЛЬБРУС ЭТО': [21, 101, 3, 1595188793],  
#     'Live: 4.08.2020 Инструктор, ма': [16, 41, 3, 1596525625],  
#     'Еще вчера этим двум ребятам Се': [10, 82, 0, 1595755712],  
#     'Начинаем набор группы в поход ': [4, 48, 1, 1595870407],  
#     'Осталось 10 туров на Эльбрус, ': [28, 399, 6, 1594321990],  
#     'Приглашаем к сотрудничеству!\n\n': [2, 54, 1, 1594208684],  
#     'Прямой эфир 21.07.2020. Инстру': [34, 93, 5, 1595317547],  
#     'РАБОТА ГОРНЫМ ГИДОМ НА ЭЛЬБРУС': [1, 44, 2, 1595698325],  
#     'СПЕЦПРЕДЛОЖЕНИЕ: Горящие туры ': [9, 802, 10, 1595515264]} 

{'Live: 4.08.2020 Инструктор, ма': [16, 52, 3, 1596525625],
 'Live: Восхождение на Эльбрус |': [3, 22, 0, 1597214895],
 'ВОСХОЖДЕНИЕ НА ЗИМНИЙ ЭЛЬБРУС\n': [6, 269, 6, 1597834786],
 'ГОРЯЩЕЕ ПРЕДЛОЖЕНИЕ!\n\nМаршрут:': [12, 300, 7, 1597000688],
 'Еще вчера этим двум ребятам Се': [10, 90, 0, 1595755712],
 'Начинаем набор группы в поход ': [5, 56, 1, 1595870407],
 'Последние 10 туров на Эльбрус ': [0, 163, 1, 1598188550],
 'РАБОТА ГОРНЫМ ГИДОМ НА ЭЛЬБРУС': [1, 49, 4, 1595698325],
 'СПЕЦПРЕДЛОЖЕНИЕ: Горящие туры ': [9, 804, 10, 1595515264]}


#### Задание 1 - SMM index

Напишите функцию get_smm_index(group_name, token), которая по имени группы и авторизационному токену API возвращает smm_index группы - сумму лайков, комментариев и репостов для последних 10 постов, поделённую на количество участников в группе.

In [164]:
def get_smm_index(group_name, token):
    
    # Количество пользователей в группе    
    url = 'https://api.vk.com/method/groups.getMembers'  
    params = {  
        'group_id': group_name,  
        'v': 5.95,  
        'access_token': token  
    }  
    
    response = requests.get(url, params = params)  
    
    user_count = response.json()['response']['count']

    
    # Вытягиваем последние новости
    url = 'https://api.vk.com/method/wall.get'  
    params = {  
    'domain': group_name,  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
    } 
    
    response = requests.get(url, params = params)    
    
    stats = 0  
              
    for record in response.json()['response']['items'][:]:  
        stats += (record['comments']['count'] + record['likes']['count'] + record['reposts']['count'])
    
    return user_count, stats/ user_count
    
    


get_smm_index('mountelbrus', token)

(66145, 0.02905737395116789)

In [165]:
import requests 
def get_smm_index(group_name, token):
  # количество участников группы
    url_m = 'https://api.vk.com/method/groups.getMembers'  
    params = {'group_id': group_name, 'v': 5.95, 'access_token': token}  
    response_m = requests.get(url_m, params = params)  
    data_m = response_m.json()  
    members = data_m['response']['count']
  # количество лайков, комментарией, репостов в 10 сообщениях
    comments = []
    likes = []
    reposts = []
    url_s = 'https://api.vk.com/method/wall.get'  
    params = { 'domain': group_name, 'filter': 'owner','count': 10, 'offset': 0,'access_token': token,'v': 5.95}  
    response_s = requests.get(url_s, params = params)  
    data_s = response_s.json()
    data=data_s['response']
    for i in data['items']:
        likes.append(i['likes']['count'])
        reposts.append(i['reposts']['count'])
        comments.append(i['comments']['count'])
  # smm_index
    smm_index = (sum(comments) + sum(likes) + sum(reposts))/members     
    return smm_index

get_smm_index('mountelbrus', token)

0.02905737395116789

#### Возможности API

Мы рассмотрели базовое взаимодействие с пользователями и группами. ВКонтакте предоставляет достаточно широкие возможности в своём API: по сути, всё, что можно делать руками через браузер, доступно и в API. Обратим внимание: если вы размещаете рекламу в Вконтакте, то можно выгружать всю статистику через ads API.

Полный список методов можно посмотреть в документации - https://vk.com/dev/methods